In [1]:
import tika
import tika.parser
from tika import parser
import requests
import urllib.parse
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#run "python -mhttp.server 8888" in the directory where your photos are

#run ifconfig in terminal to find your ip address listed for en0

images_url_usc  = 'http://10.25.179.208:8888'

images_url_usc2 = 'http://10.25.77.81:8888'

images_url_home = 'http://192.168.1.13:8888'

images_url_MOC = 'http://10.25.188.76:8888'

# Webscraping all 80,581 URLs

In [31]:
import requests
from bs4 import BeautifulSoup
import csv

webpage_url = images_url_usc2

response = requests.get(webpage_url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')

image_urls_set = set()
count = 0
for link in soup.find_all('a'):
    href = link.get('href')
    if href is not None and href.startswith('http'):
        image_urls_set.add(href)
        count += 1
    elif href is not None:
        image_urls_set.add(webpage_url + '/' + href)
        count += 1
    print(f"Processed {count} links", end='\r')

image_urls = list(image_urls_set)

# Save the image URLs to a text file
with open('image_urls.txt', mode='w') as file:
    for url in image_urls:
        file.write(url + '\n')
        
print("All links processed!") # added for confirmation


All links processed!s


In [33]:
#verifying there were no mistakes durnig link extraction

with open('image_urls.txt', 'r') as f:
    num_links = len(f.readlines())

print(f'The file contains {num_links} links.')


with open('image_urls.txt', 'r') as f:
    unique_links = set(line.strip() for line in f if line.strip())

num_unique_links = len(unique_links)

print(f'The file contains {num_unique_links} unique links.')





The file contains 80581 links.
The file contains 80581 unique links.


# Image Captions DF

- High level strategy: create a list of key value pairs, and save it to a JSON file periodically so that if the program crashes I don't lose my progress


In [5]:
#docker run -it -p 8764:8764 uscdatascience/im2txt-rest-tika
#test_url = 'http://localhost:8764/inception/v3/caption/image?url=http://10.25.77.81:8888/Pixstory-image-164318304274781.png'

captions_results = []

# Loop through the image URLs and classify each image
for url in image_urls:

    # Construct the API URL for the image classification
    api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + url

    # Send the API request
    response = requests.get(api_captions_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the list
    captions_results.append({

        'url': url, 
        'caption':first_sentence
    })

# Create a Pandas dataframe from the list of dictionaries
captions_df = pd.DataFrame(captions_results)

# Set display options to show all columns and avoid truncation of string data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Display the dataframe
captions_df

,url,caption
0,http://192.168.1.13:8888/Pixstory-image-161668402049563.jpg,a bunch of bananas that are on a table .
1,http://192.168.1.13:8888/Pixstory-image-161681318757326.jpg,a man sitting on a couch with a laptop .
2,http://192.168.1.13:8888/Pixstory-image-161984246212221.jpeg,a man in a suit and tie holding a cell phone .
3,http://192.168.1.13:8888/Pixstory-image-162014760674045.jpeg,a bunch of birds that are flying in the air .
4,http://192.168.1.13:8888/Pixstory-image-162051250471634.jpeg,a little girl sitting in a chair holding a teddy bear .
5,http://192.168.1.13:8888/Pixstory-image-162153257364412.png,a bike parked on the side of a road .
6,http://192.168.1.13:8888/Pixstory-image-162157341332029.jpeg,a man in a suit and tie standing in a room .
7,http://192.168.1.13:8888/Pixstory-image-162194441755927.jpeg,a red fire hydrant sitting in the middle of a forest .
8,http://192.168.1.13:8888/Pixstory-image-162244448366044.jpeg,a bunch of flowers that are on a table .
9,http://192.168.1.13:8888/Pixstory-image-16227252974324.png,a close up of a person holding a pair of scissors


# Image Captions JSON file creation utilizing threading

In [8]:
import threading
import queue
import time
import json

# create a queue to hold the image URLs
image_url_queue = queue.Queue()

# populate the queue with the image URLs
for image_url in image_urls:
    image_url_queue.put(image_url)

# create a list to hold the processed image data
image_captions_data = []

# define a function to process the images
def process_images():
    while True:
        # get the next image URL from the queue
        image_url = image_url_queue.get()

        # Construct the API URL for the image classification
        api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + image_url

        # Send the API request
        response = requests.get(api_captions_url)

        # Parse the JSON response
        json_response = response.json()

        # Extract the first listed sentence
        first_sentence = json_response['captions'][0]['sentence']

        # create a dictionary to store the image URL and its caption
        data = {'url': image_url, 'caption': first_sentence}

        # add the dictionary to the list of image data
        image_captions_data.append(data)

        # save the image data to disk periodically
        if len(image_captions_data) % 100 == 0:
            with open('image_captions_data.json', 'w') as f:
                json.dump(image_captions_data, f)
            print('Saved image data to disk')

        # mark the image URL as processed
        image_url_queue.task_done()

# create a pool of worker threads to process the images
num_threads = 10
for i in range(num_threads):
    t = threading.Thread(target=process_images)
    t.daemon = True
    t.start()

# wait for all images to be processed
image_url_queue.join()

# save the final set of image data to disk
with open('image_captions_data.json', 'w') as f:
    json.dump(image_captions_data, f)
print('Saved image data to disk')


Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/pj/c3bwxcz17fx4v5_bp1h430h00000gn/T/ipykernel_7794/2652990721.py", line 47, in process_images
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/queue.py", line 75, in task_done
    raise ValueError('task_done() called too many times')
ValueError: task_done() called too many times
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/pj/c3bwxcz17fx4v5_bp1

In [7]:
import requests
import json
from queue import Queue
from threading import Thread

# load the image URLs from a file
with open('image_urls.txt') as f:
    image_urls = [line.strip() for line in f.readlines()]

# load the index of the last processed image URL from a file, or start from the beginning if no file exists
try:
    with open('last_index.txt') as f:
        last_index = int(f.read())
except FileNotFoundError:
    last_index = 0

# create a queue to hold the image URLs
image_url_queue = Queue()

# populate the queue with the image URLs
for image_url in image_urls[last_index:]:
    image_url_queue.put(image_url)

# create a list to hold the processed image data
image_captions_data = []

# define a function to process the images
def process_images():
    counter = 0
    while True:
        # get the next image URL from the queue
        image_url = image_url_queue.get()

        # construct the API URL for the image classification
        api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + image_url

        # send the API request
        response = requests.get(api_captions_url)

        # parse the JSON response
        json_response = response.json()

        # extract the first listed sentence
        first_sentence = json_response['captions'][0]['sentence']

        # create a dictionary to store the image URL and its caption
        data = {'url': image_url, 'caption': first_sentence}

        # add the dictionary to the list of image data
        image_captions_data.append(data)

        # save the image data to disk periodically
        if len(image_captions_data) % 100 == 0:
            with open('image_captions_data.json', 'w') as f:
                json.dump(image_captions_data, f)
            print('Saved image data to disk')

        # mark the image URL as processed
        image_url_queue.task_done()
        counter += 1

        # print progress message periodically
        if counter % 100 == 0:
            print(f'Processed {counter} images')

# create a pool of worker threads to process the images
num_threads = 20
for i in range(num_threads):
    t = Thread(target=process_images)
    t.daemon = True
    t.start()

# wait for all images to be processed
image_url_queue.join()

# save the final set of image data to disk
with open('image_captions_data.json', 'w') as f:
    json.dump(image_captions_data, f)
print('Saved image data to disk')

#when I use 10 threads after 7 minutes this returned 300 image captions --> 1.4 seconds per image caption --> 31 hours to get all images, too long
#second try after maxing out cpu: 5 minutes extra 300 image captions  --> 1 second per image caption --> 22 hours to get all images
#third try after using 20 threads instead of 10 4.5 minutes


Saved image data to disk
Saved image data to disk


KeyboardInterrupt: 

In [9]:
import json

# load the JSON data from file
with open('image_captions_data.json') as f:
    image_data = json.load(f)

# count the number of dictionaries in the list
num_image_captions = len(image_data)

print(f'The JSON file contains {num_image_captions} image captions.')


The JSON file contains 200 image captions.


Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk


Exception in thread Thread-44:
Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 910, in json
    return complexjson.loads(self.text, **kwargs)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/

Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Processed 100 images
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
Saved image data to disk
S

Exception in thread Thread-43:
Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 910, in json
    return complexjson.loads(self.text, **kwargs)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/

Saved image data to disk
Saved image data to disk


# extracting links from a list and using multi threading

# Step 1: output as json file that updates periodically

In [ ]:
import requests
import json
import time

# Load the image URLs from a file
with open('image_urls.txt') as f:
    image_urls = [line.strip() for line in f.readlines()]

# Set the interval for saving the image data to disk
save_interval = 100

# Initialize the list to hold the processed image data
captions_results = []

# Loop through the image URLs and classify each image
for i, url in enumerate(image_urls):
    # Construct the API URL for the image classification
    api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + url

    # Send the API request
    response = requests.get(api_captions_url)

    # Parse the JSON response
    json_response = response.json()

    # Extract the first listed sentence
    first_sentence = json_response['captions'][0]['sentence']

    # Add the results to the list
    captions_results.append({
        'url': url,
        'caption': first_sentence
    })

    # Save the image data to disk periodically
    if i % save_interval == 0:
        with open('image_captions_data.json', 'w') as f:
            json.dump(captions_results, f)
        print(f'Saved {i+1} image captions to disk')

# Save the final set of image data to disk
with open('image_captions_data.json', 'w') as f:
    json.dump(captions_results, f)
print(f'Saved {len(image_urls)} image captions to disk')
